In [1]:
import ee
import geemap
import colorcet as cc

In [2]:
ee.Authenticate()

True

In [3]:
ee.Initialize(project="hotspotstoplight")

Maybe update this to use five year summer median composite, per this workflow? Has really good references: https://google-earth-engine.com/Human-Applications/Heat-Islands/

In [4]:
aoi = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1").filter(
    ee.Filter.eq("ADM1_NAME", "Daerah Istimewa Yogyakarta")
)

In [5]:
startDate = "2023-06-01"
endDate = "2023-09-21"

In [6]:
# Applies scaling factors.
def apply_scale_factors(image):
    # Scale and offset values for optical bands
    optical_bands = image.select("SR_B.").multiply(0.0000275).add(-0.2)

    # Scale and offset values for thermal bands
    thermal_bands = image.select("ST_B.*").multiply(0.00341802).add(149.0)

    # Add scaled bands to the original image
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)


# Function to Mask Clouds and Cloud Shadows in Landsat 8 Imagery
def cloud_mask(image):
    # Define cloud shadow and cloud bitmasks (Bits 3 and 5)
    cloud_shadow_bitmask = 1 << 3
    cloud_bitmask = 1 << 5

    # Select the Quality Assessment (QA) band for pixel quality information
    qa = image.select("QA_PIXEL")

    # Create a binary mask to identify clear conditions (both cloud and cloud shadow bits set to 0)
    mask = (
        qa.bitwiseAnd(cloud_shadow_bitmask)
        .eq(0)
        .And(qa.bitwiseAnd(cloud_bitmask).eq(0))
    )

    # Update the original image, masking out cloud and cloud shadow-affected pixels
    return image.updateMask(mask)

In [7]:
# Import and preprocess Landsat 8 imagery
image = (
    ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
    .filterBounds(aoi)
    .filterDate(startDate, endDate)
    .map(apply_scale_factors)
    .map(cloud_mask)
    .median()
    .clip(aoi)
)

# Calculate Normalized Difference Vegetation Index (NDVI)
ndvi = image.normalizedDifference(["SR_B5", "SR_B4"]).rename("NDVI")

# Calculate the minimum and maximum NDVI value within the AOI
ndvi_min = ee.Number(ndvi.reduceRegion(ee.Reducer.min(), aoi, 30).values().get(0))
ndvi_max = ee.Number(ndvi.reduceRegion(ee.Reducer.max(), aoi, 30).values().get(0))

# Fraction of Vegetation (FV) Calculation
fv = ndvi.subtract(ndvi_min).divide(ndvi_max.subtract(ndvi_min)).pow(2).rename("FV")

# Emissivity Calculation
em = fv.multiply(ee.Number(0.004)).add(ee.Number(0.986)).rename("EM")

# Select Thermal Band (Band 10) and Rename It
thermal = image.select("ST_B10").rename("thermal")

# Land Surface Temperature (LST) Calculation
lst = thermal.expression(
    "(TB / (1 + (0.00115 * (TB / 1.438)) * log(em))) - 273.15",
    {
        "TB": thermal.select("thermal"),  # Select the thermal band
        "em": em,  # Assign emissivity
    },
).rename("LST")

In [9]:
Map = geemap.Map()
Map.addLayer(aoi, {}, "AOI - Yogyakarta")
Map.centerObject(aoi, 10)

# Define visualization parameters for True Color imagery (bands 4, 3, and 2)
visualization = {
    "bands": ["SR_B4", "SR_B3", "SR_B2"],
    "min": 0.0,
    "max": 0.15,
}

Map.addLayer(image, visualization, "True Color 432")

# Define NDVI Visualization Parameters
ndvi_palette = {"min": -1, "max": 1, "palette": ["blue", "white", "green"]}

Map.addLayer(ndvi, ndvi_palette, "NDVI")

# Add the LST Layer to the Map with Custom Visualization Parameters
Map.addLayer(
    lst,
    {
        "min": 18.47,
        "max": 42.86,
        "palette": [
            "040274",
            "040281",
            "0502a3",
            "0502b8",
            "0502ce",
            "0502e6",
            "0602ff",
            "235cb1",
            "307ef3",
            "269db1",
            "30c8e2",
            "32d3ef",
            "3be285",
            "3ff38f",
            "86e26f",
            "3ae237",
            "b5e22e",
            "d6e21f",
            "fff705",
            "ffd611",
            "ffb613",
            "ff8b13",
            "ff6e08",
            "ff500d",
            "ff0000",
            "de0101",
            "c21301",
            "a71001",
            "911003",
        ],
    },
    "Land Surface Temperature 2023",
)
Map

Map(center=[-7.894577385263086, 110.4473379167677], controls=(WidgetControl(options=['position', 'transparent_…